카드 짝 맞추기

In [ ]:
from collections import defaultdict,deque
from itertools import permutations

class ftn:
    def __init__(self,board,r,c):
        s = (r,c)
        dic = defaultdict(list)
        for r in range(4):
            for c in range(4):
                if board[r][c] :
                    dic[board[r][c]].append((r,c))

        dic = {k:[[v[0],v[1]],[v[1],v[0]]] for k,v in dic.items()}

        self.ALL = []

        pos = list(permutations(dic.keys(),len(dic.keys())))
        for pp in pos:
            tmp = [[s]]
            for p in pp:
                tmp = [x + dic[p][i] for x in tmp for i in range(2)]
            self.ALL += tmp
        
        self.ans = float('INF')
        self.base = len(dic) * 2
    
    def move(self,n,p,arr):
        px,py = abs(n[0] - p[0]),abs(n[1] - p[1])
        cnt = [0,0]
        # 위 아래 먼저 움직임 : idx 0
        # 좌 우 먼저 움직임 : idx 1
        if px < 2:
            cnt[0] += px;cnt[1] += px
        elif px == 3:
            cnt[0] += sum(1 for x in arr if min(n[0],p[0]) < x[0] < max(n[0],p[0]) and x[1] == n[1]) + 1
            cnt[1] += sum(1 for x in arr if min(n[0],p[0]) < x[0] < max(n[0],p[0]) and x[1] == p[1]) + 1
        else: # px == 2
            cnt[0] += 1 if ((p[0],n[1]) in arr or p[0] in (0,3)) and (min(p[0],n[0])+1,n[1]) not in arr else 2
            cnt[1] += 1 if (min(p[0],n[0])+1,p[1]) not in arr else 2
        # print(cnt)
        if py < 2:
            cnt[0] += py;cnt[1] += py
        elif py == 3:
            cnt[0] += sum(1 for x in arr if min(n[1],p[1]) < x[1] < max(n[1],p[1]) and x[0] == p[0]) + 1
            cnt[1] += sum(1 for x in arr if min(n[1],p[1]) < x[1] < max(n[1],p[1]) and x[0] == n[0]) + 1
        else: # py == 2
            cnt[0] += 1 if (p[0],min(p[1],n[1])+1) not in arr else 2
            cnt[1] += 1 if ((n[0],p[1]) in arr or p[1] in (0,3)) and (n[0],min(n[1],p[1])+1) not in arr else 2
        # print(cnt)
        return min(cnt)

def solution(board, r, c):
    sol = ftn(board,r,c)
    
    while sol.ALL:
        tmp = deque(sol.ALL.pop())
        s,e = tmp.popleft(),tmp.popleft()
        cnt = sol.base
        while 1:
            cnt += sol.move(s,e,tmp)
            if tmp:
                s,e = e, tmp.popleft()
            else:
                break
            if cnt > sol.ans:
                break
        if sol.ans > cnt:
            sol.ans = cnt
    
    return sol.ans


외벽 점검

In [251]:
# 모든 경우의 수 다 확인
from collections import deque
def solution(n, weak, dist):
    dist.sort(key = lambda x:-x)
    visited = {}
    que = deque([tuple(weak)])

    for idx,d in enumerate(dist):
        next_que = deque([])
        while que:
            q = que.popleft()

            for i in q:
                if i + d < n:
                    tmp = tuple(x for x in q if x < i or i+d < x)
                else:
                    tmp = tuple(x for x in q if x < i and x+n > i+d)
                if len(tmp) == 0:
                    return idx + 1
                elif tmp not in visited:
                    visited[tmp] = 1
                    next_que.append(tmp)
                
                if i - d >= 0:
                    tmp = tuple(x for x in q if x < i-d or x > i)
                else:
                    tmp = tuple(x for x in q if i < x < n - i + d)

                if len(tmp) == 0:
                    return idx + 1
                elif tmp not in visited:
                    visited[tmp] = 1
                    next_que.append(tmp)
        que = next_que
    return -1

In [245]:
# 14번 틀림
# 한번 시계방향으로 돌면 계속 시계방향으로 돌아야하지 않나?
def solution(n, weak, dist):
    dist.sort(key = lambda x:-x)
    pos = []

    for w in weak:
        pos.append(sorted([ww - w if ww >= w else n-w+ww for ww in weak]))
        pos.append(sorted([n + w - ww if ww > w else w - ww for ww in weak]))
        
    for idx,d in enumerate(dist):
        for p_idx,p in enumerate(pos):
            cnt = (0,15,None)
            for x_idx,x in enumerate(p):
                tmp = [xx for xx in p if x+d<xx or xx<x]
                
                if len(tmp) < cnt[1]:
                    cnt = (x_idx,len(tmp),tmp)

            if cnt[1] == 0:
                return idx +1
            
            else:
                pos[p_idx] = cnt[-1]
                
    return -1

매칭 점수

In [ ]:
import re
def solution(word, pages):
    Link = r'\<{} [a-z|A-Z|0-9|\:\\\/\.\"\' \=]*\>'
    url = re.compile(r'https://[a-z|A-Z|0-9|\:\\\/\.\']*')
    body = re.compile(r'\<body\>[a-z|A-Z|0-9|\n\t\\\<\> \=\'\":\/\\\.!@#\$\%\^\&\*\(\)\[\]\{\}\+\-\,]*\<\/body\>')
    db = re.compile(r'(\<[a-z|A-Z|0-9|\:\\\/\.\"\' \=]*\>)|\n|\t')
    alpha = re.compile(r'[^a-z|A-Z]+')

    dic = {}

    for idx,p in enumerate(pages):
        URL = url.search(re.search(Link.format('meta'),p).group()).group()
        dic[URL] = {'idx':idx,'lscore':0}

        dic[URL]['link'] = [url.search(x).group() for x in re.findall(Link.format('a'),p)]
        txt = alpha.sub(" ",db.sub(" ", body.search(p).group())).lower()
        dic[URL]['score'] = sum(x == word.lower() for x in txt.split(" "))
    
    for k,v in dic.items():
        for dv in v['link']:
            if dv in dic:
                dic[dv]['lscore'] += v['score']/len(v['link'])
    return sorted(dic.values(),key = lambda v : (-(v['lscore'] + v['score']),v['idx']))[0]['idx']


110 옮기기

In [250]:
from collections import deque
import re
def ftn(s):
    que,l,tmp = deque(s),deque([]),deque([])
    cnt = 0
    while 1:
        tmp.append(que.popleft())

        if len(tmp) == 3:
            if tmp == deque(['1','1','0']):
                cnt += 1
                tmp = []
                while l:
                    tmp += l.pop()
                    if len(tmp) == 2:
                        break
                tmp = deque(tmp[::-1])
            else:
                l.append(tmp.popleft())

        if len(que) == 0:
            s = ""
            while l :
                s += l.popleft()
            while tmp:
                s += tmp.popleft()
            break

    tmp = re.search('1{2}',s)
    if tmp:
        return s[:tmp.span()[0]] + "110" * cnt + s[tmp.span()[0]:]
    else:

        return s[:s.rfind('0')+1] + '110' * cnt + s[s.rfind('0')+1:]
           

def solution(s):
    return [ftn(x) for x in s]

단속 카메라

In [ ]:
def solution(routes):
    arr = []
    routes.sort(key = lambda x:x[1])
    
    for r in routes:
        if len(arr) == 0:
            arr.append(r[1])
        else:
            for ar in arr:
                if r[0]<=ar<=r[1]:
                    break
            else:
                arr.append(r[1])
    return len(arr)

블록 이동하기

In [ ]:
from collections import deque
class ftn:
    def __init__(self,board):
        self.dic = {((0,0),(0,1)) : 0}
        self.N = len(board)
        self.board = board
        self.que = deque([(((0,0),(0,1)),0)])
    
    def poss(self,coord):
        (x1,y1),(x2,y2) = coord
        pos = []
        if x1 == x2 : # 가로
            if 0<=y2+1<self.N and self.board[x1][y2+1] == 0: # 좌
                pos.append(((x2,y2),(x2,y2+1)))
            
            if 0<=y1-1<self.N and self.board[x1][y1-1] == 0: # 우
                pos.append(((x1,y1-1),(x1,y1)))
            
            if 0<=x1-1<self.N and self.board[x1-1][y1] == 0 and self.board[x1-1][y2] == 0: # 위 + 회전
                pos.append(((x1-1,y1),(x1-1,y2)))
                pos.append(((x1-1,y1),(x1,y1)))
                pos.append(((x1-1,y2),(x2,y2)))

            if 0<=x1+1<self.N and self.board[x1+1][y1] == 0 and self.board[x1+1][y2] == 0: # 아래 + 회전
                
                pos.append(((x1+1,y1),(x1+1,y2)))
                pos.append(((x1,y1),(x1+1,y1)))
                pos.append(((x2,y2),(x2+1,y2)))
            
        else: # 세로
            if 0<=x2+1<self.N and self.board[x2+1][y2] == 0: #아래
                pos.append(((x2,y2),(x2+1,y2)))
                
            if 0<=x1-1<self.N and self.board[x1-1][y1] == 0: #위
                pos.append(((x1-1,y1),(x1,y1)))
                
            if 0<=y1-1<self.N and self.board[x1][y1-1] == 0 and self.board[x2][y2-1] == 0: # 좌 + 회전
                pos.append(((x1,y1-1),(x2,y2-1)))
                pos.append(((x1,y1-1),(x1,y1)))
                pos.append(((x2,y2-1),(x2,y2)))
                
            if 0<=y1+1<self.N and self.board[x1][y1+1] == 0 and self.board[x2][y2+1] == 0: # 우 + 회전
                pos.append(((x1,y1+1),(x2,y2+1)))
                pos.append(((x1,y1),(x1,y1+1)))
                pos.append(((x2,y2),(x2,y2+1)))
                
        return pos
            
                
def solution(board):
    sol = ftn(board)
    while sol.que:
        coord,cnt = sol.que.popleft()
        poss = sol.poss(coord)
        
        for p in poss:
            if (sol.N-1,sol.N-1) in p :
                return cnt + 1
            
            if sol.dic.get(p,float("INF")) > cnt + 1:
                sol.dic[p] = cnt + 1
                sol.que.append((p,cnt + 1))

풍선 터뜨리기

In [249]:
from collections import deque
def solution(a):
    l,r = [],[]
    for t in a:
        if len(l) == 0 or l[-1] > t:
            l.append(t)
        else:
            l.append(l[-1])
            
    for t in a[::-1]:
        if len(r) == 0 or r[-1] > t:
            r.append(t)
        else:
            r.append(r[-1])
    l,a,r = deque(l),deque(a),deque(r)
    
    ans = 0
    
    while l:
        tl,ta,tr = l.popleft(),a.popleft(),r.pop()
        if max(tl,ta,tr) != ta or tr == ta or tl == ta:
            ans += 1
    return ans